In [ ]:
# !pip install tensorflow

# !pip install opencv-python

# !pip install pandas

# !pip install matplotlib

# !pip install -U scikit-learn

# !pip install imgaug

# !pip install numpy

## Imports

In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
from sklearn.utils import shuffle

### Function to import image files

In [ ]:
def getName(filePath):
    return filePath.split('\\')[-1]

def importDataInfo(path):
    columns = ['Center', 'Left', 'Right', 'Steering', 'Throttle', 'Brake', 'Speed']
    data = pd.read_csv(os.path.join(path, 'driving_log.csv'), names = columns)
#### REMOVE FILE PATH AND GET ONLY FILE NAME
#     print(data.head())    
#     print(data['Center'][0])
#     print(getName(data['Center'][0]))
    data['Center']=data['Center'].apply(getName)
    print(data.head())
    print('Total Images Imported',data.shape[0])
    return data

In [ ]:
path = 'myData'
data = importDataInfo(path)

## Visualization and Exploration

In [ ]:
def balanceData(data,display=True):
    nBin = 30
    samplesPerBin = 1000
    hist, bins = np.histogram(data['Steering'], nBin)
#     print(bins)
    
#     center = (bins[:-1] + bins[1:]) * 0.5
#     print(center)
#     plt.bar(center, hist, width = 0.6)
#     plt.plot((-1, 1), (samplesPerBin, samplesPerBin))
#     plt.show()

    if display:
        center = (bins[:-1] + bins[1:]) * 0.5
        plt.bar(center, hist, width=0.06)
        plt.xlabel('Steering angle')
        plt.ylabel('Samples per Bin')
        plt.ylim(0, 1500)
        plt.plot((np.min(data['Steering']), np.max(data['Steering'])), (samplesPerBin, samplesPerBin))
        plt.show()
        
    removeindexList = []
    for j in range(nBin):
        binDataList = []
        for i in range(len(data['Steering'])):
            if data['Steering'][i] >= bins[j] and data['Steering'][i] <= bins[j + 1]:
                binDataList.append(i)
        binDataList = shuffle(binDataList)
        binDataList = binDataList[samplesPerBin:]
        removeindexList.extend(binDataList)

    print('Removed Images:', len(removeindexList))
    data.drop(data.index[removeindexList], inplace=True)
    print('Remaining Images:', len(data))
    
    if display:
        hist, _ = np.histogram(data['Steering'], (nBin))
        plt.bar(center, hist, width=0.06)
        plt.plot((np.min(data['Steering']), np.max(data['Steering'])), (samplesPerBin, samplesPerBin))
        plt.show()
        
    return data

In [ ]:
balanceData(data)

In [ ]:
# data = balanceData(data, display=False)

## Data Preprocessing

In [ ]:
def loadData(path, data):
    imagesPath = []
    steering = []
    
    for i in range(len(data)):
        indexed_data = data.iloc[i]
        imagesPath.append(f'{path}/IMG/{indexed_data[0]}')
        steering.append(float(indexed_data[3]))
    imagesPath = np.asarray(imagesPath)
    steering = np.asarray(steering)
    return imagesPath, steering

In [ ]:
imagesPath, steerings = loadData(path,data)
# print(imagesPath[0], steerings[0])

### Split data into training and testing

In [ ]:
from sklearn.model_selection import train_test_split

xTrain, xVal, yTrain, yVal = train_test_split(imagesPath, steerings, test_size=0.2,random_state=10)
print('Total Training Images: ',len(xTrain))
print('Total Validation Images: ',len(xVal))

### Data Augmentation

In [ ]:
import matplotlib.image as mpimg
from imgaug import augmenters as iaa
import cv2

In [ ]:
def augmentImage(imgPath,steering):
    img =  mpimg.imread(imgPath)
    # Pan
    if np.random.rand() < 0.5:
        pan = iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)})
        img = pan.augment_image(img)
    # Zoom
    if np.random.rand() < 0.5:
        zoom = iaa.Affine(scale=(1, 1.2))
        img = zoom.augment_image(img)
    # Brigtness
    if np.random.rand() < 0.5:
        brightness = iaa.Multiply((0.2, 1.2))
        img = brightness.augment_image(img)
    # Flip
    if np.random.rand() < 0.5:
        img = cv2.flip(img, 1)
        steering = -steering
    return img, steering

In [ ]:
# imgRe, st = augmentImage('test.jpg', 0)
# plt.imshow(imgRe)
# plt.show()

### Preprocessing for training

In [ ]:
def preProcess(img):
    img = img[60:135,:,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img

In [ ]:
imgRe = preProcess(mpimg.imread('test.jpg'))
plt.imshow(imgRe)
plt.show()

### Batch size

In [ ]:
import random

In [ ]:
def batchGen(imagesPath, steeringList, batchSize, trainFlag):
    while True:
        imgBatch = []
        steeringBatch = []

        for i in range(batchSize):
            index = random.randint(0, len(imagesPath) - 1)
            if trainFlag:
                img, steering = augmentImage(imagesPath[index], steeringList[index])
            else:
                img = mpimg.imread(imagesPath[index])
                steering = steeringList[index]
            img = preProcess(img)
            imgBatch.append(img)
            steeringBatch.append(steering)
        yield (np.asarray(imgBatch),np.asarray(steeringBatch))

## Creating model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
def createModel():
    model = Sequential()
    
    model.add(Convolution2D(24, (5, 5), (2, 2), input_shape=(66, 200, 3), activation='elu'))
    model.add(Convolution2D(36, (5, 5), (2, 2), activation='elu'))
    model.add(Convolution2D(48, (5, 5), (2, 2), activation='elu'))
    model.add(Convolution2D(64, (3, 3), activation='elu'))
    model.add(Convolution2D(64, (3, 3), activation='elu'))

    model.add(Flatten())
    model.add(Dense(100, activation = 'elu'))
    model.add(Dense(50, activation = 'elu'))
    model.add(Dense(10, activation = 'elu'))
    model.add(Dense(1))

    model.compile(Adam(learning_rate=0.0001),loss='mse')
    return model

In [ ]:
model = createModel()
model.summary()

### Model Training

In [ ]:
history = model.fit(batchGen(xTrain, yTrain, 10, 1),
                  steps_per_epoch=20,
                  epochs=2,
                  validation_data=batchGen(xVal, yVal, 10, 0),
                  validation_steps=20)

In [ ]:
history = model.fit(batchGen(xTrain, yTrain, 100, 1),
                      steps_per_epoch=300,
                      epochs=10,
                      validation_data=batchGen(xVal, yVal, 100, 0),
                      validation_steps=200)

In [ ]:
model.save('model.h5')
print('Model Saved')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])
plt.ylim(0, 0.2)
plt.title('Loss')
plt.xlabel('Epoch')
plt.show()